In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [89]:
headers = 'Pass your header here'
url = 'https://www.ambitionbox.com/list-of-companies?page={page}'

def get_webpage(page=1):
    print(page, end=' ')
    response = requests.get(url=url.format(page=page),
                           headers=headers)
    
    if response.status_code != 200:
        return False
    return response        

In [90]:
def get_page_content(soup):
    company_data = {
        'name': [],
        'rating': [],
        'icon_domain': [],
        'location': [],
        'age': [],
        'size': [],
        'tags': [],
    }
    
    all_company_div = soup.find_all('div', {'itemprop': 'itemListElement'})

    for company_div in all_company_div:
        # Company name
        company_data['name'].append(company_div.find('h2').text.strip())
        
        # Company rating
        company_data['rating'].append(company_div.find('p', {'class': 'rating'}).text.strip())

        # Company icon domain
        icon_domain = company_div.find('i', {'class': 'icon-domain'})
        company_data['icon_domain'].append(icon_domain.parent.text.strip() if icon_domain else None) 
    
        # Company location
        location = company_div.find('i', {'class': 'icon-pin-drop'})
        company_data['location'].append(location.parent.text.strip() if location else None) 
    
        # Company age
        age = company_div.find('i', {'class': 'icon-access-time'})
        company_data['age'].append(age.parent.text.strip() if age else None) 
    
        # Company size
        size = company_div.find('i', {'class': 'icon-supervisor-account'})
        company_data['size'].append(size.parent.text.strip() if size else None) 

        # Company tags
        company_tags = company_div.find('ul', {'class': 'chips-block'})
        if company_tags:
            tag_string = ','.join(map(lambda x:x.text.strip(), company_tags.find_all('li')))
            company_data['tags'].append(tag_string)
        else:
            company_data['tags'].append('')

    company_df = pd.DataFrame(company_data)  
    return company_df      

In [91]:
counter = 1
final_df = pd.DataFrame()

while True:
    webpage = get_webpage(counter)
    counter += 1
    
    if webpage == False:
        break

    soup = BeautifulSoup(webpage.content, 'html.parser')
    page_df = get_page_content(soup)
    final_df = pd.concat([final_df, page_df])
    

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [92]:
final_df.shape

(9989, 7)

In [93]:
final_df.head()

,name,rating,icon_domain,location,age,size,tags
0,TCS,3.9,Public,"Mumbai,Maharashtra + 271 more",55 years old,1 Lakh+ Employees (India),"BPO,IT Services & Consulting,Forbes Global 200..."
1,Accenture,4.1,Public,Dublin + 131 more,34 years old,1 Lakh+ Employees (India),"IT Services & Consulting,Software Product,Forb..."
2,Cognizant,3.9,Private,Teaneck. New Jersey. + 101 more,29 years old,1 Lakh+ Employees (India),"IT Services & Consulting,Forbes Global 2000"
3,ICICI Bank,4.0,Public,"Mumbai,Maharashtra + 1059 more",29 years old,1 Lakh+ Employees (India),"Financial Services,Banking,Forbes Global 2000,..."
4,HDFC Bank,4.0,Public,"Mumbai,Maharashtra + 1270 more",29 years old,50k-1 Lakh Employees (India),"Financial Services,Banking,Insurance,Forbes Gl..."
